# Packages

In [1]:
from PIL import Image
import os
from datasets import Dataset
import datasets
from transformers import AutoImageProcessor
from transformers import DefaultDataCollator
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import tensorflow as tf
import evaluate
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer


In [2]:
from huggingface_hub import notebook_login

notebook_login()

# Load Data

In [3]:
def load_images_from_folder(folder_path):
    images = []
    labels = []

    for root, dirs, files in os.walk(folder_path):
        for subfolder in dirs:
            subfolder_path = os.path.join(root, subfolder)
            
            for filename in os.listdir(subfolder_path):
                image_path = os.path.join(subfolder_path, filename)
                
                try:
                    # Open the image using PIL
                    image = Image.open(image_path)
                    images.append(image)
                    labels.append(subfolder)
                    
                    # Create a dictionary with image and label
                    data = {'image': images, 'label': labels} # Dict to store the lists with images and labels

                except Exception as e:
                    print(f"Error processing {image_path}: {e}")

    return data

# Replace 'path_to_images_folder' with the path to your folder containing subfolders 'yes' and 'no'
folder_path = 'test_data'
image_data = load_images_from_folder(folder_path)
image_data['label'] = [1 if label == 'yes' else 0 for label in image_data['label']]


Error processing test_data\train\no: [Errno 13] Permission denied: 'test_data\\train\\no'
Error processing test_data\train\yes: [Errno 13] Permission denied: 'test_data\\train\\yes'


In [4]:
image_data = Dataset.from_dict(image_data)
image_data = image_data.train_test_split(test_size=0.2)

In [5]:
labels = [sample['label'] for sample in image_data['train']]
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

# Preprocess

In [6]:
checkpoint = "google/vit-base-patch16-224-in21k"
image_processor = AutoImageProcessor.from_pretrained(checkpoint)

In [7]:
from torchvision.transforms import RandomResizedCrop, Compose, Normalize, ToTensor

normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
size = (
    image_processor.size["shortest_edge"]
    if "shortest_edge" in image_processor.size
    else (image_processor.size["height"], image_processor.size["width"])
)
_transforms = Compose([RandomResizedCrop(size), ToTensor(), normalize])

def transforms(examples):
    examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["image"]]
    del examples["image"]
    return examples

image_data.with_transform(transforms)

data_collator = DefaultDataCollator()


# Evaluate

In [8]:
accuracy = evaluate.load("accuracy")

In [9]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

# Train

In [10]:
model = AutoModelForImageClassification.from_pretrained(
    checkpoint,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id,
)

training_args = TrainingArguments(
    output_dir="Spam_classification_model_huggingface",
    remove_unused_columns=False,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=image_data["train"],
    eval_dataset=image_data["test"],
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
)

trainer.train()

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/9 [00:00<?, ?it/s]

RuntimeError: Could not infer dtype of JpegImageFile

# Inference Pytorch